In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


When we created our baseline linear regression model, we excluded a considerable minority of the data because of the amount of missing and unusable values in the raw weather data.  While we were still pursing several strategies for filling in these gaps with sensible values, we felt that it would be prudent to examine classification algrithms that would be robust to these missing values to cover our bases in case the weather imputation strategies ran into trouble.  Also, seeing these challenges in the development enviornments lead us to believe that these challenges may be even more severe in the production enviorment, so being resilant to these issues would be beneficial.  At the same time the linear regression model had many important qualities we didn't want to lose, such as explainability and a natural approach to feature reduction.  Decision trees stood out as an algorithm that has one of the best mix of the benefits and data resilience that seemed appropriate to the problem at hand.

Decision trees classify a dataset by find a value in one of the parameters that can be used to divide the entire set into two groups.  Then each group is subsequently split over and over again, until all the remaining groups are pure, meaning that all of the samples in that group have the same classification.  Determining the optimal split point is achieved by trying to make each group as pure as possible, and the measure of purity is the gini index.  It is deviation from a perfectly equal distribution, expressed as \\(Gini Index = 1-\sum_{i=1}^{n} p_i^2\\) where \\(p\\) is the probability of each class.
This approach can have a tendency to follow the training data too closely thought, which is overfitting, so we are adopting the Random Forests variant of Decisions Trees, which builds a variety of trees from different random subsets of the training data, a process known as bagging.  Additionally, the trees are trained on random subsets of the features, which is known as boosting. When a prediction is made, each of the trees is evaluated and a majority vote becomes the decision.

Below we have created a small example of how a Decision Tree would handle a sample of data with potientially missing weather values.  This produces a model that has high explainablity and an easy way to view feature importance so we will be able to prune are model for more parsimony.

In [0]:
column_names =  ['ORIGIN','DEST','DEP_HOUR','ARR_HOUR','AVG_TMP_DEG_ORIGIN', 'DEP_DEL15']

In [0]:
train = spark.read.parquet('dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/train')

In [0]:
data_with_nulls = train.filter('ORIGIN IN ("ATL","ORD") AND DEST IN ("ORD","ATL")').select('ORIGIN','DEST','DEP_HOUR','ARR_HOUR','AVG_TMP_DEG_ORIGIN','DEP_DEL15').limit(12)
# data_without_nulls = train.filter('AVG_TMP_DEG_ORIGIN is not null AND ORIGIN = "ATL"').select('ORIGIN','DEST','DEP_HOUR','ARR_HOUR','AVG_TMP_DEG_ORIGIN','DEP_DEL15').limit(5)
# data_without_nulls = data_without_nulls.union(train.filter('AVG_TMP_DEG_ORIGIN is not null AND ORIGIN = "ORD"').select('ORIGIN','DEST','DEP_HOUR','ARR_HOUR','AVG_TMP_DEG_ORIGIN','DEP_DEL15').limit(5))

In [0]:
data_with_nulls.display()

ORIGIN DEST DEP_HOUR ARR_HOUR AVG_TMP_DEG_ORIGIN DEP_DEL15 ATL ORD 8 9 -4.390625 0.0 ATL ORD 8 9 151.23295454545453 0.0 ATL ORD 7 8 227.30935251798562 0.0 ATL ORD 7 8 222.35365853658536 0.0 ATL ORD 21 22 179.94405594405595 0.0 ATL ORD 7 8 -126.57377049180327 0.0 ATL ORD 13 14 185.01503759398497 0.0 ATL ORD 21 22 188.0 1.0 ATL ORD 13 14 252.6086956521739 0.0 ATL ORD 21 22 209.60833333333332 0.0 ATL ORD 6 7 220.31944444444446 1.0 ATL ORD 7 8 176.7536231884058 0.0

In [0]:
training_data = [
    ['ATL', 'ORD', 8, 9, -4.390625, 0.0],
    ['ATL', 'ORD', 8, 9, 151.23295454545453, 0.0],
    ['ATL', 'ORD', 21, 22, None, 0.0],
    ['ATL', 'ORD', 21, 22, 187.9034090909091, 1.0],
    ['ATL', 'ORD', 13, 14, 252.6086956521739, 0.0],
    ['ATL', 'ORD', 6, 7, 220.31944444444446, 1.0],
    ['ORD', 'ATL', 18, 21, 314.2142857142857, 0.0],
    ['ORD', 'ATL', 18, 21, 285.17083333333335, 1.0],
    ['ORD', 'ATL', 9, 12, None, 1.0],
    ['ORD', 'ATL', 20, 23, 266.3333333333333, 0.0],
    ['ORD', 'ATL', 12, 15, 190.712, 0.0],
    ['ORD', 'ATL', 6, 9, 21.32089552238806, 0.0]
]

In [0]:
training_data_df = pd.DataFrame(training_data, columns=column_names)
training_data_df

Out[87]:

ORIGIN 
 DEST 
 DEP_HOUR 
 ARR_HOUR 
 AVG_TMP_DEG_ORIGIN 
 DEP_DEL15 
 
 
 
 
 0 
 ATL 
 ORD 
 8 
 9 
 -4.390625 
 0.0 
 
 
 1 
 ATL 
 ORD 
 8 
 9 
 151.232955 
 0.0 
 
 
 2 
 ATL 
 ORD 
 21 
 22 
 0.000000 
 0.0 
 
 
 3 
 ATL 
 ORD 
 21 
 22 
 187.903409 
 1.0 
 
 
 4 
 ATL 
 ORD 
 13 
 14 
 252.608696 
 0.0 
 
 
 5 
 ATL 
 ORD 
 6 
 7 
 220.319444 
 1.0 
 
 
 6 
 ORD 
 ATL 
 18 
 21 
 314.214286 
 0.0 
 
 
 7 
 ORD 
 ATL 
 18 
 21 
 285.170833 
 1.0 
 
 
 8 
 ORD 
 ATL 
 9 
 12 
 0.000000 
 1.0 
 
 
 9 
 ORD 
 ATL 
 20 
 23 
 266.333333 
 0.0 
 
 
 10 
 ORD 
 ATL 
 12 
 15 
 190.712000 
 0.0 
 
 
 11 
 ORD 
 ATL 
 6 
 9 
 21.320896 
 0.0

In [0]:
training_data_with_nulls = [
    ['ATL', 'DCA', 9, 11, 80.5530303030303, 0.0],
    ['ATL', 'MIA', 7, 8, 53.05590062111801, 1.0],
    ['ATL', 'DFW', 6, 7, 166.61111111111111, 0.0],
    ['ATL', 'LAX', 12, 14, 187.9034090909091, 0.0],
    ['ATL', 'LAX', 9, 12, 65.44029850746269, 1.0],
    ['ATL', 'MIA', 6, 8, 70.97058823529412, 0.0],
    ['ATL', 'LAX', 18, 20, 0, 0.0],
    ['ATL', 'MIA', 7, 8, 0, 0.0],
    ['ORD', 'JAC', 11, 14, 212.52091254752852, 0.0],
    ['ORD', 'ABE', 17, 18, 190.84615384615384, 1.0],
    ['ORD', 'MIA', 6, 7, 190.92307692307693, 0.0],
    ['ORD', 'DFW', 19, 21, 11.9296875, 0.0],
    ['ORD', 'JAC', 10, 12, 211.88212927756655, 0.0],
    ['ORD', 'DFW', 12, 15, 0, 0.0],
    ['ORD', 'EGE', 10, 12, 22.759036144578314, 0.0],
    ['ORD', 'MIA', 13, 17, 0, 1.0]
]

In [0]:
X = pd.get_dummies(training_data_df.drop('DEP_DEL15',axis=1))
y = training_data_df[['DEP_DEL15']]

In [0]:
dtree = DecisionTreeClassifier()
dtree = dtree.fit(X,y)

In [0]:
export_graphviz(dtree, '/dbfs/mnt/mids-w261/team20SSDK/images/tree.out', feature_names=X.columns.values)

In [0]:
df = sc.parallelize(training_data_with_nulls).toDF(column_names)
df.show()

+------+----+--------+--------+------------------+---------+
ORIGIN|DEST|DEP_HOUR|ARR_HOUR|AVG_TMP_DEG_ORIGIN|DEP_DEL15|
+------+----+--------+--------+------------------+---------+
 ATL| DCA| 9| 11| 80.5530303030303| 0.0|
 ATL| MIA| 7| 8| 53.05590062111801| 1.0|
 ATL| DFW| 6| 7|166.61111111111111| 0.0|
 ATL| LAX| 12| 14| 187.9034090909091| 0.0|
 ATL| LAX| 9| 12| 65.44029850746269| 1.0|
 ATL| MIA| 6| 8| 70.97058823529412| 0.0|
 ATL| LAX| 18| 20| null| 0.0|
 ATL| MIA| 7| 8| null| 0.0|
 ORD| JAC| 11| 14|212.52091254752852| 0.0|
 ORD| ABE| 17| 18|190.84615384615384| 1.0|
 ORD| MIA| 6| 7|190.92307692307693| 0.0|
 ORD| DFW| 19| 21| 11.9296875| 0.0|
 ORD| JAC| 10| 12|211.88212927756655| 0.0|
 ORD| DFW| 12| 15| null| 0.0|
 ORD| EGE| 10| 12|22.759036144578314| 0.0|
 ORD| MIA| 13| 17| null| 1.0|
+------+----+--------+--------+------------------+---------+

In [0]:
#from pyspark.mllib.regression import LabeledPoint

origin = {"ATL": 0.0, "ORD": 1.0}
dest = {"DCA": 0.0, "MIA": 1.0, "DFW": 2.0, "LAX": 3.0, "JAC": 4.0, "ABE": 5.0, "EGE": 6.0}

labeledpoints = [
  
  LabeledPoint(0.0, [origin["ATL"], dest['DCA'], 9,11,80.5530303030303]),
  LabeledPoint(1.0, [origin["ATL"], dest['MIA'], 7, 8, 53.05590062111801]),
  LabeledPoint(0.0, [origin["ATL"], dest['DFW'], 6, 7, 166.61111111111111]),
  LabeledPoint(0.0, [origin["ATL"], dest['LAX'], 12, 14, 187.9034090909091]),
  LabeledPoint(1.0, [origin["ATL"], dest['LAX'], 9, 12, 65.44029850746269]),
  LabeledPoint(0.0, [origin["ATL"], dest['MIA'], 6, 8, 70.97058823529412]),
  LabeledPoint(0.0, [origin["ATL"], dest['LAX'], 18, 20, None]),
  LabeledPoint(0.0, [origin["ATL"], dest['MIA'], 7, 8, None]),
  LabeledPoint(0.0, [origin["ORD"], dest['JAC'], 11, 14, 212.52091254752852]),
  LabeledPoint(1.0, [origin["ORD"], dest['ABE'], 17, 18, 190.84615384615384]),
  LabeledPoint(0.0, [origin["ORD"], dest['MIA'], 6, 7, 190.92307692307693]),
  LabeledPoint(0.0, [origin["ORD"], dest['DFW'], 19, 21, 11.9296875]),
  LabeledPoint(0.0, [origin["ORD"], dest['JAC'], 10, 12, 211.88212927756655]),
  LabeledPoint(0.0, [origin["ORD"], dest['DFW'], 12, 15, None]),
  LabeledPoint(0.0, [origin["ORD"], dest['EGE'], 10, 12, 22.759036144578314]),
  LabeledPoint(1.0, [origin["ORD"], dest['MIA'], 13, 17, None]),
]

data = sc.parallelize(labeledpoints)

In [0]:
from pyspark.mllib.regression import LabeledPoint

origin = {"ATL": 0.0, "ORD": 1.0}
dest = {"ATL": 0.0, "ORD": 1.0}
#dest = {"DCA": 0.0, "MIA": 1.0, "DFW": 2.0, "LAX": 3.0, "JAC": 4.0, "ABE": 5.0, "EGE": 6.0}

labeledpoints = [
  
#   ['ATL', 'ORD', 8, 9, -4.390625, 0.0],
#     ['ATL', 'ORD', 8, 9, 151.23295454545453, 0.0],
#     ['ATL', 'ORD', 21, 22, 179.94405594405595, 0.0],
#     ['ATL', 'ORD', 21, 22, 187.9034090909091, 1.0],
#     ['ATL', 'ORD', 13, 14, 252.6086956521739, 0.0],
#     ['ATL', 'ORD', 6, 7, 220.31944444444446, 1.0],
#     ['ORD', 'ATL', 18, 21, 314.2142857142857, 0.0],
#     ['ORD', 'ATL', 18, 21, 285.17083333333335, 1.0],
#     ['ORD', 'ATL', 9, 12, 220.94814814814814, 1.0],
#     ['ORD', 'ATL', 20, 23, 266.3333333333333, 0.0],
#     ['ORD', 'ATL', 12, 15, 190.712, 0.0],
#     ['ORD', 'ATL', 6, 9, 21.32089552238806, 0.0]
  
  LabeledPoint(0.0, [origin["ATL"], dest['ORD'], 8, 9, -4.390625]),
  LabeledPoint(0.0, [origin["ATL"], dest['ORD'], 8, 9, 151.23295454545453]),
  LabeledPoint(0.0, [origin["ATL"], dest['ORD'], 21, 22, None]),
  LabeledPoint(1.0, [origin["ATL"], dest['ORD'], 21, 22, 187.9034090909091]),
  LabeledPoint(0.0, [origin["ATL"], dest['ORD'], 13, 14, 252.6086956521739]),
  LabeledPoint(1.0, [origin["ATL"], dest['ORD'], 6, 7, 220.31944444444446]),
  LabeledPoint(0.0, [origin["ORD"], dest['ATL'], 18, 21, 314.2142857142857]),
  LabeledPoint(1.0, [origin["ORD"], dest['ATL'], 18, 21, 285.17083333333335]),
  LabeledPoint(1.0, [origin["ORD"], dest['ATL'], 9, 12, None]),
  LabeledPoint(0.0, [origin["ORD"], dest['ATL'], 20, 23, 266.3333333333333]),
  LabeledPoint(0.0, [origin["ORD"], dest['ATL'], 12, 15, 190.712]),
  LabeledPoint(0.0, [origin["ORD"], dest['ATL'], 6, 9, 21.32089552238806])
]

data = sc.parallelize(labeledpoints)

In [0]:
from pyspark.mllib.tree import DecisionTree

model = DecisionTree.trainClassifier(data=data,

         numClasses=2,

         categoricalFeaturesInfo={0: 2, 1:7})

In [0]:
print(model.toDebugString())

DecisionTreeModel classifier of depth 4 with 11 nodes
 If (feature 3 <= 8.0)
 Predict: 1.0
 Else (feature 3 > 8.0)
 If (feature 4 <= 268.8897644927536)
 If (feature 2 <= 20.5)
 Predict: 0.0
 Else (feature 2 > 20.5)
 Predict: 1.0
 Else (feature 4 > 268.8897644927536)
 If (feature 0 in {0.0})
 Predict: 0.0
 Else (feature 0 not in {0.0})
 If (feature 2 <= 10.5)
 Predict: 1.0
 Else (feature 2 > 10.5)
 Predict: 0.0

In [0]:
# -----

In [0]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [0]:
#######
# Demo:
unique_vals(training_data, 1)
# unique_vals(training_data, 1)
#######

Out[23]: {'ABE', 'DCA', 'DFW', 'EGE', 'JAC', 'LAX', 'MIA'}

In [0]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [0]:
#######
# Demo:
class_counts(training_data)
#######

Out[26]: {0.0: 9, 1.0: 3}

In [0]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [0]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

Out[29]: True

In [0]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            column_names[self.column], condition, str(self.value))

In [0]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(2, 3)

Out[34]: Is DEP_HOUR >= 3?

In [0]:
# How about one for a categorical attribute
q = Question(0, 'ATL')
q


Out[38]: Is ORIGIN == ATL?

In [0]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

Out[39]: True

In [0]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [0]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'ORD'))
# This will contain all the 'Red' rows.
true_rows

Out[42]: [['ORD', 'JAC', 11, 14, 212.52091254752852, 0.0],
 ['ORD', 'ABE', 17, 18, 190.84615384615384, 1.0],
 ['ORD', 'MIA', 6, 7, 190.92307692307693, 0.0],
 ['ORD', 'DFW', 19, 21, 11.9296875, 0.0],
 ['ORD', 'JAC', 10, 12, 211.88212927756655, 0.0],
 ['ORD', 'EGE', 10, 12, 22.759036144578314, 0.0]]

In [0]:
# This will contain everything else.
false_rows
#######

Out[43]: [['ATL', 'DCA', 9, 11, 80.5530303030303, 0.0],
 ['ATL', 'MIA', 7, 8, 53.05590062111801, 1.0],
 ['ATL', 'DFW', 6, 7, 166.61111111111111, 0.0],
 ['ATL', 'LAX', 12, 14, 187.9034090909091, 0.0],
 ['ATL', 'LAX', 9, 12, 65.44029850746269, 1.0],
 ['ATL', 'MIA', 6, 8, 70.97058823529412, 0.0]]

In [0]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [0]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

Out[45]: 0.0

In [0]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

Out[46]: 0.5

In [0]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

Out[48]: 0.7999999999999998

In [0]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [0]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

Out[50]: 0.375

In [0]:
# How much information do we gain by partioning on 'ATL'?
true_rows, false_rows = partition(training_data, Question(1, 'EGE'))
info_gain(true_rows, false_rows, current_uncertainty)


Out[55]: 0.011363636363636354

In [0]:
# What about if we partioned on 'ORD' instead?
true_rows, false_rows = partition(training_data, Question(1,'LAX'))
info_gain(true_rows, false_rows, current_uncertainty)

Out[54]: 0.025000000000000133

In [0]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'ATL'))

# Here, the true_rows contain only 'Grapes'.
true_rows

Out[57]: [['ATL', 'DCA', 9, 11, 80.5530303030303, 0.0],
 ['ATL', 'MIA', 7, 8, 53.05590062111801, 1.0],
 ['ATL', 'DFW', 6, 7, 166.61111111111111, 0.0],
 ['ATL', 'LAX', 12, 14, 187.9034090909091, 0.0],
 ['ATL', 'LAX', 9, 12, 65.44029850746269, 1.0],
 ['ATL', 'MIA', 6, 8, 70.97058823529412, 0.0]]

In [0]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question